In [1]:
import ccxt
import math
from datetime import datetime
import numpy as np
import re

In [2]:
def fetch_exchange(exch_name, exch):
    graph = {}
    # load markets
    market = exch.load_markets(True)

    if (exch.has['fetchTickers']):
        exch_tickers = exch.fetch_tickers()
        for symbol in exch_tickers.keys():
            try:
                node_to, node_from = symbol.split('/')
                try:
                    w_to = -math.log(1 / float(exch_tickers[symbol]['info']['askPrice']))
                    w_from = -math.log(float(exch_tickers[symbol]['info']['askPrice']))
                    if node_from not in graph:
                        graph[node_from] = {}
                    graph[node_from][node_to] = {'weight': w_to, 'd': 'direct'}
                    if node_to not in graph:
                        graph[node_to] = {}
                    graph[node_to][node_from] = {'weight': w_from, 'd': 'reverse'}
                except:
                    pass
            except:
                print('symbol error')
    return graph

In [3]:
def initialize(graph, source):
    d = {} # Stands for destination
    p = {} # Stands for predecessor
    for node in graph:
        d[node] = float('Inf') # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0 # For the source we know how to reach
    return d, p
 
def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]['weight']:
        # Record this lower distance
        d[neighbour]  = d[node] + graph[node][neighbour]['weight']
        p[neighbour] = node
        
def retrace_negative_loop(p, start):
    
    arbitrageLoop = []
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            reversed_arbitrageLoop = arbitrageLoop[::-1]
            return reversed_arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1): #Run this until is converges
        for u in graph:
            for v in graph[u]: #For each neighbour of u
                #relax(u, v, graph, d, p) #Lets relax it
                if d[v] > d[u] + graph[u][v]['weight']:
                    # Record this lower distance
                    d[v]  = d[u] + graph[u][v]['weight']
                    p[v] = u
    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] > d[u] + graph[u][v]['weight']:
                return(retrace_negative_loop(p, source))
    return None


def collect_negative_cycle():
    binance = ccxt.binance({
    'apiKey': 'y',
    'secret': 'Y', })
    
    paths = []
    graph = fetch_exchange('binance', binance)
    #graph = {'USDT': {'BTC': {'weight': 18000}}, 'BTC': {'ETH': {'weight': -18000}}, 'ETH':{'USDT':{'weight': -100}}}
    
    path = bellman_ford(graph, 'USDT')
    if path not in paths and not None:
        paths.append(path)

    for path in paths:
        if path == None:
            print("No opportunity here :(")
        else:
            print(path)
            graph_sum = 0
            for i in range(len(path) - 1):
                print(graph[path[i]][path[i + 1]]['weight'])
                graph_sum += graph[path[i]][path[i + 1]]['weight']
            print('total sum:')
            print(graph_sum)
            print(math.exp(-graph_sum))
    

In [4]:
%%time
collect_negative_cycle()

['AUD', 'USDT', 'WNXM', 'BUSD', 'SXP', 'AUD']
0.3182237575644797
3.3285191561641696
-3.36719237635947
0.0812112554248232
-0.4007875185570534
total sum:
-0.04002572576305086
1.0408375501881608
CPU times: user 325 ms, sys: 17.8 ms, total: 343 ms
Wall time: 1.56 s
